In [1]:
import pandas as pd
import re
import zhon.hanzi
from collections import Counter
# import yaml as yaml

In [40]:
zh_pat = f"[{zhon.hanzi.characters}|{zhon.hanzi.punctuation}]"

with open("./虐童案/prepare/stop_word_chinese_punc.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[0:]

stopwords[0:20]

['圖／',
 '文／',
 '／綜合報導',
 '記者',
 '攝',
 '提供',
 '／台北報導',
 ',',
 '\\u3000',
 '?',
 '、',
 '。',
 '“',
 '”',
 '，',
 '。',
 '！',
 '？',
 '；',
 '：']

In [4]:
zh_pat

'[〇一-\u9fff㐀-䶿豈-\ufaff𠀀-\U0002a6df𪜀-\U0002b73f𫝀-\U0002b81f丽-\U0002fa1f|＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､\u3000、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏﹑﹔·！？｡。]'

In [141]:
#CKIP
import torch

## Content segment

In [13]:
import unicodedata
def remove_punc_by_unicode(words):
    out = []
    for word in words:
        if word != " " and not unicodedata.category(word[0]).startswith('P'):
            out.append(word)
    return out

In [36]:
# 定義移除stopwords的函數
def remove_stopwords(text):
    TT = text
    for i in stopwords:
        TT = TT.replace(i, "")
    return TT

### data

In [41]:
df = pd.read_csv("./寶林茶室/data/寶林_ettoday_all.csv")[['date', 'title', 'article']]
df.head(2)

,date,title,article
0,2024-03-29 21:18:00,網傳寶林清庫存釀禍？ 最新進貨日期曝,▲寶林茶室負責人黎仿軒。（圖／記者劉昌松攝） 文／CTWANT 寶林茶室中毒案延燒，目前已...
1,2024-03-29 20:30:00,8人驗出米酵菌酸「食材陰性」王必勝要民眾放心：2情況鎖定未擴散,記者鄒鎮宇／綜合報導 北市寶林茶室信義A13店爆發「米酵菌酸」食安風波，累計24名消費者疑似...


In [43]:
df['article_removePunc'] = df['article'].apply(remove_stopwords)
df

,date,title,article,article_removePunc
0,2024-03-29 21:18:00,網傳寶林清庫存釀禍？ 最新進貨日期曝,▲寶林茶室負責人黎仿軒。（圖／記者劉昌松攝） 文／CTWANT 寶林茶室中毒案延燒，目前已...,寶林茶室負責人黎仿軒劉昌松 CTWANT 寶林茶室中毒案延燒目前已造成2人死亡檢警後續也在...
1,2024-03-29 20:30:00,8人驗出米酵菌酸「食材陰性」王必勝要民眾放心：2情況鎖定未擴散,記者鄒鎮宇／綜合報導 北市寶林茶室信義A13店爆發「米酵菌酸」食安風波，累計24名消費者疑似...,鄒鎮宇 北市寶林茶室信義A13店爆發米酵菌酸食安風波累計24名消費者疑似吃到有疑慮的炒粿條和...
2,2024-03-29 20:19:00,快訊／「大食代」捲寶林風暴 新安東京：若確定是食品中毒將理賠,▲信義A13寶林茶室爆發食安風暴，大、小房東所投保的國泰產險和新安東京雙雙點頭理賠。（圖／記...,信義A13寶林茶室爆發食安風暴大小房東所投保的國泰產險和新安東京雙雙點頭理賠袁茵 柯沛辰 台...
3,2024-03-29 20:15:00,寶林重症者全驗出米酵菌酸！食材卻陰性 王必勝：粿條仍相當可疑,記者洪巧藍／台北報導 寶林茶室食品中毒案再有進展！衛福部今（29）日晚間公布最新檢驗結果，包...,洪巧藍 寶林茶室食品中毒案再有進展衛福部今29日晚間公布最新檢驗結果包含死者住院病患共8人驗...
4,2024-03-29 19:24:00,寶林中毒案被告增至4人！正班廚師首傳訊 檢諭知限制出境出海,記者劉昌松／台北報導 北市寶林茶室信義A13店爆發「米酵菌酸」食安風波，累計24名消費者疑似...,劉昌松 北市寶林茶室信義A13店爆發米酵菌酸食安風波累計24名消費者疑似吃到有疑慮的炒粿條和...
...,...,...,...,...
364,2024-03-28 6:21:00,快訊／寶林茶室「關鍵證物」凌晨查扣 39歲行銷鬼才吃炒粿條不治,記者葉國吏／綜合報導 遠百信義A13美食街寶林茶室疑似食物中毒事件延燒，截至目前為止共12...,葉國吏 遠百信義A13美食街寶林茶室疑似食物中毒事件延燒截至目前為止共12人受害其中已經造成...
365,2024-03-28 6:07:00,妹吃「寶林茶室」肝腎衰竭！姊姊捐肝搶命 憂3／31後找嘸人負責,▲信義A13寶林茶室。（圖／記者袁茵攝） 記者葉國吏／綜合報導 遠百信義A13美食街寶林茶...,信義A13寶林茶室袁茵 葉國吏 遠百信義A13美食街寶林茶室疑似食物中毒事件延燒截至目前為止...
366,2024-03-28 6:03:00,寶林案元凶偏向米酵菌酸！醫：攻擊粒線體害全身 感染食物下肚無解藥,▲信義A13寶林茶室出現集體食物中毒事件。（資料照／記者袁茵攝） 記者張乃文／台北報導 台北...,信義A13寶林茶室出現集體食物中毒事件資料照／袁茵 張乃台北報導 台北市知名素食餐廳寶林茶室...
367,2024-03-28 5:57:00,「吃個炒粿條人就沒了」寶林茶室第2死 女兒淚訴：原本還能說笑,記者葉國吏／綜合報導 信義A13「寶林茶室」食物中毒風波持續延燒，66歲楊姓重症病患宣告不...,葉國吏 信義A13寶林茶室食物中毒風波持續延燒66歲楊姓重症病患宣告不治楊男女兒痛哭還原事發...


In [44]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [45]:
# ws_driver  = CkipWordSegmenter(model="bert-base", device=torch.device('mps')) # 用GPU會出下面那句error
ws_driver  = CkipWordSegmenter(model="bert-base", device=-1)
# pos_driver = CkipPosTagger(model="bert-base", device=-1)



In [46]:
df['tokens']  = ws_driver(df['article_removePunc'])

Inference: 100%|██████████████████████████████████████████████████████████████████████████| 3/3 [05:16<00:00, 105.34s/it]


In [49]:
def check_empty(list_tokens):
    tok = []
    for t in list_tokens:
        if t == "" or t == " ": continue
        tok.append(t)
    return tok
df['token_check'] = df['tokens'].apply(check_empty)
df

,date,title,article,article_removePunc,tokens,token_check
0,2024-03-29 21:18:00,網傳寶林清庫存釀禍？ 最新進貨日期曝,▲寶林茶室負責人黎仿軒。（圖／記者劉昌松攝） 文／CTWANT 寶林茶室中毒案延燒，目前已...,寶林茶室負責人黎仿軒劉昌松 CTWANT 寶林茶室中毒案延燒目前已造成2人死亡檢警後續也在...,"[ , 寶林, 茶室, 負責人, 黎仿軒, 劉昌松, CTWANT , 寶林, 茶室, 中...","[寶林, 茶室, 負責人, 黎仿軒, 劉昌松, CTWANT , 寶林, 茶室, 中毒案,..."
1,2024-03-29 20:30:00,8人驗出米酵菌酸「食材陰性」王必勝要民眾放心：2情況鎖定未擴散,記者鄒鎮宇／綜合報導 北市寶林茶室信義A13店爆發「米酵菌酸」食安風波，累計24名消費者疑似...,鄒鎮宇 北市寶林茶室信義A13店爆發米酵菌酸食安風波累計24名消費者疑似吃到有疑慮的炒粿條和...,"[鄒鎮宇, , 北市, 寶林, 茶室, 信義, A13, 店, 爆發, 米酵菌酸, 食安,...","[鄒鎮宇, 北市, 寶林, 茶室, 信義, A13, 店, 爆發, 米酵菌酸, 食安, 風波..."
2,2024-03-29 20:19:00,快訊／「大食代」捲寶林風暴 新安東京：若確定是食品中毒將理賠,▲信義A13寶林茶室爆發食安風暴，大、小房東所投保的國泰產險和新安東京雙雙點頭理賠。（圖／記...,信義A13寶林茶室爆發食安風暴大小房東所投保的國泰產險和新安東京雙雙點頭理賠袁茵 柯沛辰 台...,"[信義, A13, 寶林, 茶室, 爆發, 食安, 風暴, 大小, 房東, 所, 投保, 的...","[信義, A13, 寶林, 茶室, 爆發, 食安, 風暴, 大小, 房東, 所, 投保, 的..."
3,2024-03-29 20:15:00,寶林重症者全驗出米酵菌酸！食材卻陰性 王必勝：粿條仍相當可疑,記者洪巧藍／台北報導 寶林茶室食品中毒案再有進展！衛福部今（29）日晚間公布最新檢驗結果，包...,洪巧藍 寶林茶室食品中毒案再有進展衛福部今29日晚間公布最新檢驗結果包含死者住院病患共8人驗...,"[洪巧藍, , 寶林, 茶室, 食品, 中毒案, 再, 有, 進展, 衛福部, 今, 29...","[洪巧藍, 寶林, 茶室, 食品, 中毒案, 再, 有, 進展, 衛福部, 今, 29日, ..."
4,2024-03-29 19:24:00,寶林中毒案被告增至4人！正班廚師首傳訊 檢諭知限制出境出海,記者劉昌松／台北報導 北市寶林茶室信義A13店爆發「米酵菌酸」食安風波，累計24名消費者疑似...,劉昌松 北市寶林茶室信義A13店爆發米酵菌酸食安風波累計24名消費者疑似吃到有疑慮的炒粿條和...,"[劉昌松, , 北市, 寶林, 茶室, 信義, A13店, 爆發, 米酵菌酸, 食安, 風...","[劉昌松, 北市, 寶林, 茶室, 信義, A13店, 爆發, 米酵菌酸, 食安, 風波, ..."
...,...,...,...,...,...,...
364,2024-03-28 6:21:00,快訊／寶林茶室「關鍵證物」凌晨查扣 39歲行銷鬼才吃炒粿條不治,記者葉國吏／綜合報導 遠百信義A13美食街寶林茶室疑似食物中毒事件延燒，截至目前為止共12...,葉國吏 遠百信義A13美食街寶林茶室疑似食物中毒事件延燒截至目前為止共12人受害其中已經造成...,"[葉國吏, , 遠百, 信義, A13, 美食街, 寶林, 茶室, 疑似, 食物, 中毒,...","[葉國吏, 遠百, 信義, A13, 美食街, 寶林, 茶室, 疑似, 食物, 中毒, 事件..."
365,2024-03-28 6:07:00,妹吃「寶林茶室」肝腎衰竭！姊姊捐肝搶命 憂3／31後找嘸人負責,▲信義A13寶林茶室。（圖／記者袁茵攝） 記者葉國吏／綜合報導 遠百信義A13美食街寶林茶...,信義A13寶林茶室袁茵 葉國吏 遠百信義A13美食街寶林茶室疑似食物中毒事件延燒截至目前為止...,"[信義, A13, 寶林, 茶室, 袁茵 , 葉國吏, , 遠百, 信義, A13, 美食...","[信義, A13, 寶林, 茶室, 袁茵 , 葉國吏, 遠百, 信義, A13, 美食街, ..."
366,2024-03-28 6:03:00,寶林案元凶偏向米酵菌酸！醫：攻擊粒線體害全身 感染食物下肚無解藥,▲信義A13寶林茶室出現集體食物中毒事件。（資料照／記者袁茵攝） 記者張乃文／台北報導 台北...,信義A13寶林茶室出現集體食物中毒事件資料照／袁茵 張乃台北報導 台北市知名素食餐廳寶林茶室...,"[信義, A13, 寶林, 茶室, 出現, 集體, 食物, 中毒, 事件, 資料, 照, ／...","[信義, A13, 寶林, 茶室, 出現, 集體, 食物, 中毒, 事件, 資料, 照, ／..."
367,2024-03-28 5:57:00,「吃個炒粿條人就沒了」寶林茶室第2死 女兒淚訴：原本還能說笑,記者葉國吏／綜合報導 信義A13「寶林茶室」食物中毒風波持續延燒，66歲楊姓重症病患宣告不...,葉國吏 信義A13寶林茶室食物中毒風波持續延燒66歲楊姓重症病患宣告不治楊男女兒痛哭還原事發...,"[葉國吏, , 信義, A13, 寶林, 茶室, 食物, 中毒, 風波, 持續, 延燒, ...","[葉國吏, 信義, A13, 寶林, 茶室, 食物, 中毒, 風波, 持續, 延燒, 66,..."


In [51]:
df.to_pickle("./寶林茶室/BAK/article_token.pkl")